# IMPORTING PACKAGES 📦📦

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm, skew
from scipy import stats 
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
import math
from sklearn.metrics import accuracy_score
import random
random.seed(42)

# DATA OVERVIEWS 🧐📊

In [ ]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
sample_submission = pd.read_csv("../input/titanic/gender_submission.csv")

In [ ]:
train_df.head(2)

In [ ]:
test_df.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
train_df.info()

In [ ]:
column_names = train_df.columns
column_names

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=['O'])

In [ ]:
concatenated_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
concatenated_df.head(2)

In [ ]:
y_train = train_df["Survived"]
y_train.head(2)

In [ ]:
# Calculate the number of missing values in each column of the 'train_df' DataFrame
missing_values = concatenated_df.isna().sum()

# Filter columns where the count of missing values is greater than 0
# This creates a boolean mask where 'True' indicates columns with missing values
missing_values_greater_than_zero = missing_values[missing_values > 0]

# Print or use the resulting Series to see columns with missing values and their respective counts
print(missing_values_greater_than_zero)

In [ ]:
# List of features to be dropped
features_to_drop = ["PassengerId"]

# Drop the specified features from the concatenated DataFrame
concatenated_df.drop(features_to_drop, axis=1, inplace=True)

# Now, concatenated_df does not contain the specified features
concatenated_df.head(2)

# ANALYZING AND TRANSFORMING THE DATA 🔄🔍

In [ ]:
custom_colors = ['#FF5733', '#FFC300', '#FF5733', '#C70039', '#900C3F', '#581845', 
                 '#FFC300', '#DAF7A6', '#1287A5', '#F4D03F', '#00D2D3', '#FF5733', 
                 '#900C3F', '#581845', '#FFC300', '#DAF7A6', '#1287A5', '#F4D03F', 
                 '#00D2D3', '#FF5733']

# Analyzing Passenger Names 🧐👩‍✈️✨

In [ ]:
# Extract titles from the 'Name' column and create a new column 'surname'
concatenated_df['surname'] = concatenated_df['Name'].str.extract(r',\s*([^\.]+)\s*\.', expand=False)

In [ ]:
# Mapping for specific titles
title_mapping = {
    'Mr': 'Mr',
    'Mrs': 'Mrs',
    'Miss': 'Miss',
    'Master': 'Master'
}

# Create the new column 'new_surname' based on the mapping
concatenated_df['new_surname'] = concatenated_df['surname'].map(title_mapping).fillna('Others')

In [ ]:
# List of features to be dropped
features_to_drop = ["Name","surname"]

# Drop the specified features from the concatenated DataFrame
concatenated_df.drop(features_to_drop, axis=1, inplace=True)

# Now, concatenated_df does not contain the specified features
concatenated_df.head(2)

In [ ]:
# Calculate average age for each new_surname group
average_age_by_surname = concatenated_df.groupby('new_surname')['Age'].mean().round().astype(int)

# Fill missing age values with average age based on new_surname
concatenated_df['new_Age'] = concatenated_df.apply(lambda row: average_age_by_surname[row['new_surname']] 
                                                if pd.isnull(row['Age']) 
                                                else row['Age'], axis=1)

In [ ]:
# List of features to be dropped
features_to_drop = ["Age" , "new_surname"]

# Drop the specified features from the concatenated DataFrame
concatenated_df.drop(features_to_drop, axis=1, inplace=True)

# Now, concatenated_df does not contain the specified features
concatenated_df.head(2)

# Embarked Port Analysis 🌍🚢

In [ ]:
# Calculate the overall average fare
average_fare = concatenated_df['Fare'].mean()

# Fill null 'Fare' values with the overall average fare
concatenated_df['Fare'] = concatenated_df['Fare'].fillna(average_fare/5)

In [ ]:
# Group by 'Embarked' and calculate frequency and mean of 'Fare'
embarked_stats = concatenated_df.groupby('Embarked').agg({'Embarked': 'count', 'Fare': 'mean'}).rename(columns={'Embarked': 'Frequency', 'Fare': 'Mean Fare'})

# Print the frequency and mean fare for each 'Embarked' value
print(embarked_stats)

In [ ]:
# Find rows where 'Fare' column is null
rows_with_null_fare = concatenated_df[concatenated_df['Embarked'].isnull()]

# Print the rows with null 'Fare'
print(rows_with_null_fare)

In [ ]:
# Fill null values in 'Embarked' column with 'C'
concatenated_df['Embarked'] = concatenated_df['Embarked'].fillna('C')

# Dropping Ticket Column ❌🎫

In [ ]:
concatenated_df.drop('Ticket', axis=1, inplace=True)
concatenated_df.head(2)

# Processing Cabin Data 🏠🏠

In [ ]:
# Create a new feature indicating whether Cabin information is missing or not
concatenated_df['Cabin_Missing'] = concatenated_df['Cabin'].isnull().astype(int)

# Extract the first letter of the Cabin as a new feature (if available), replace NaN and 'T' with 'Z'
concatenated_df['Cabin_Letter'] = concatenated_df['Cabin'].str[0].apply(lambda x: 'Z' if pd.isnull(x) or x == 'T' else x)

In [ ]:
# Calculate average SalePrice for each MSZoning category
average_cabin = concatenated_df.groupby('Cabin_Letter')['Survived'].mean().sort_values(ascending=False)

# Bar plot to visualize average SalePrice for each MSZoning category
plt.figure(figsize=(6, 4))
average_cabin.plot(kind='bar', color=custom_colors)
plt.title('Average Survived by Cabin Letter')
plt.xlabel('Cabin Letter')
plt.ylabel('Average Survived')
plt.xticks(rotation=45)
plt.show()

# Summary statistics for SalePrice grouped by MSZoning
summary_stats = concatenated_df.groupby('Cabin_Letter')['Survived'].describe()
print(summary_stats)

In [ ]:
# List of features to be dropped
features_to_drop = ["Cabin","Cabin_Missing"]

# Drop the specified features from the concatenated DataFrame
concatenated_df.drop(features_to_drop, axis=1, inplace=True)

In [ ]:
# concatenated_df["Cabin_Letter"].head(50)
# Create binary columns for each unique letter in the "Cabin_Letter" column
cabin_letter_dummies = pd.get_dummies(concatenated_df["Cabin_Letter"])

# Rename the columns to indicate the presence of specific letters
cabin_letter_dummies.columns = ["Cabin_" + letter for letter in cabin_letter_dummies.columns]

# Concatenate the binary columns with the original DataFrame
concatenated_df = pd.concat([concatenated_df, cabin_letter_dummies], axis=1)

In [ ]:
# Drop the original "Cabin_Letter" column
concatenated_df.drop("Cabin_Letter", axis=1, inplace=True)

In [ ]:
concatenated_df.head(2)

# Analyzing Pclass 🎫🔢

In [ ]:
# Calculate average SalePrice for each MSZoning category
average_pClass = concatenated_df.groupby('Pclass')['Survived'].mean().sort_values(ascending=False)

# Bar plot to visualize average SalePrice for each MSZoning category
plt.figure(figsize=(6, 4))
average_pClass.plot(kind='bar', color=custom_colors)
plt.title('Average Survived by Pclass')
plt.xlabel('P class')
plt.ylabel('Average Survived')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Create binary columns for each unique class in the "Pclass" column
pclass_dummies = pd.get_dummies(concatenated_df["Pclass"])

# Rename the columns to indicate the presence of specific classes
pclass_dummies.columns = ["PClass_" + str(class_num) for class_num in pclass_dummies.columns]

# Concatenate the binary columns with the original DataFrame
concatenated_df = pd.concat([concatenated_df, pclass_dummies], axis=1)

# Drop the original "Pclass" column
concatenated_df.drop("Pclass", axis=1, inplace=True)

# Print the updated DataFrame
concatenated_df.head(2)

# Analyzing Embarked Data 🌍🌍

In [ ]:
# Calculate average SalePrice for each MSZoning category
average_Embarked = concatenated_df.groupby('Embarked')['Survived'].mean().sort_values(ascending=False)

# Bar plot to visualize average SalePrice for each MSZoning category
plt.figure(figsize=(6, 4))
average_Embarked.plot(kind='bar', color=custom_colors)
plt.title('Average Survived by Embarked')
plt.xlabel('Embarked')
plt.ylabel('Average Survived')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Create binary columns for each unique embarkation point in the "Embarked" column
embarked_dummies = pd.get_dummies(concatenated_df["Embarked"])

# Rename the columns to indicate the presence of specific embarkation points
embarked_dummies.columns = ["Embarked_" + embarkation_point for embarkation_point in embarked_dummies.columns]

# Concatenate the binary columns with the original DataFrame
concatenated_df = pd.concat([concatenated_df, embarked_dummies], axis=1)

# Drop the original "Embarked" column
concatenated_df.drop("Embarked", axis=1, inplace=True)

# Print the updated DataFrame
concatenated_df.head(2)

# Data Transformation (LabelEncoder) 🛠️🔤

In [ ]:
label_encoder = LabelEncoder()

categorical_columns = concatenated_df.select_dtypes(include=['O']).columns.tolist()

for column in categorical_columns:
    concatenated_df[column] = label_encoder.fit_transform(concatenated_df[column])
    
concatenated_df.head(2)

# Analyzing Age Data 📊👶👵

In [ ]:
# Plot a histogram of the 'Age' column
plt.figure(figsize=(6, 4))
plt.hist(concatenated_df["new_Age"], bins=30, color='skyblue', edgecolor='black')
plt.title('Number of People vs. Age')
plt.xlabel('Age')
plt.ylabel('Number of People')
plt.grid(True)
plt.show()

In [ ]:
# Define bins and labels for age groups
bins = [0, 10, 20, 30, 45, 60, float('inf')]
labels = [0, 1, 2, 3, 2, 1]

# Create a new column 'AgeGroup' based on the 'Age' column
concatenated_df['AgeGroup'] = pd.cut(concatenated_df['new_Age'], bins=bins, labels=labels, right=False,ordered=False)

concatenated_df.drop('new_Age', axis=1, inplace=True)

# Print the DataFrame with the new 'AgeGroup' column
concatenated_df.head(2)

In [ ]:
average_AgeGroup = concatenated_df.groupby('AgeGroup')['Survived'].mean().sort_values(ascending=False)
# Plot a histogram of the 'Age' column

plt.figure(figsize=(6, 4))
average_AgeGroup.plot(kind='bar', color=custom_colors)
plt.title('Number of People vs. AgeGroup')
plt.xlabel('Age')
plt.ylabel('Number of People')
plt.grid(True)
plt.show()

# Family Relationship Transformation 🤝💼

In [ ]:
# Convert 'SibSp' and 'Parch' based on specified conditions
concatenated_df['SibSp'] = concatenated_df['SibSp'].apply(lambda x: 0 if x == 0 else 2 if x in [1, 2] else 1)
concatenated_df['Parch'] = concatenated_df['Parch'].apply(lambda x: 0 if x == 0 else 2 if x in [1, 2] else 1)

# Print the updated DataFrame
concatenated_df.head(2)

# Exploring Fare Prices 💸🎫📊

In [ ]:
# Convert the fare data to a NumPy array
fare_data = np.array(concatenated_df["Fare"])
print(fare_data)
# Calculate WCSS for different values of k
wcss = []
for i in range(1, 15):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(fare_data.reshape(-1, 1))
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method graph
plt.figure(figsize=(6, 4))
plt.plot(range(1, 15), wcss, marker='o', linestyle='--',color="red")
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')  # Within-cluster sum of squares
plt.xticks(np.arange(1, 15, 1))
plt.show()

In [ ]:
# Convert the fare data to a NumPy array
fare_data = np.array(concatenated_df["Fare"])

# Reshape the fare data to be a 2D array
fare_data_reshaped = fare_data.reshape(-1, 1)

# Define the optimal number of clusters (found using the Elbow Method)
optimal_clusters = 5

# Apply KMeans clustering with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
cluster_labels = kmeans.fit_predict(fare_data_reshaped)

# Visualize the clusters and data points
plt.figure(figsize=(6, 4))
for i in range(optimal_clusters):
    plt.scatter(fare_data_reshaped[cluster_labels == i], [i] * len(fare_data_reshaped[cluster_labels == i]), label=f'Cluster {i+1}')

plt.scatter(kmeans.cluster_centers_, np.arange(optimal_clusters), color='red', marker='X', label='Centroids')
plt.xlabel('Fare')
plt.ylabel('Cluster')
plt.title('Clustering of Fare Data')
plt.legend()
plt.show()

# Output the cluster labels for each data point
print("Cluster Labels:")
print(cluster_labels)

In [ ]:
# Assuming you have determined the optimal number of clusters (optimal_clusters) using the Elbow Method
optimal_clusters = 5  # Replace this with the actual optimal number of clusters

# Fare data (assuming you already have this data)
fare_data = np.array(concatenated_df["Fare"])

# Reshape the fare data to be a 2D array
fare_data_reshaped = fare_data.reshape(-1, 1)

# Apply KMeans clustering with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
cluster_labels = kmeans.fit_predict(fare_data_reshaped)

# Add the cluster labels as a new column to the DataFrame
concatenated_df["Fare_Cluster"] = cluster_labels

In [ ]:
# List of features to be dropped
features_to_drop = ["Fare"]

# Drop the specified features from the concatenated DataFrame
concatenated_df.drop(features_to_drop, axis=1, inplace=True)

# Now, concatenated_df does not contain the specified features
concatenated_df.head(2)

# Bar Charts and Pie for All Columns 📊📈

In [ ]:
# Assuming univalue_columns is a list of column names containing unique values
columns_list = concatenated_df.columns

# Set up subplots
fig, axs = plt.subplots(len(columns_list), 2, figsize=(16, 4 * len(columns_list)))

for i, column in enumerate(columns_list):
    count_labels = concatenated_df[column].value_counts()

    # Plot pie chart
    axs[i, 0].pie(count_labels, labels=count_labels.index, autopct='%.1f%%', shadow=True)
    axs[i, 0].set_title(f'Pie Chart of {column} Counts')

    # Plot countplot
    sns.countplot(data=concatenated_df, x=column, hue='Survived', palette='pastel', ax=axs[i, 1])
    axs[i, 1].set_title(f'Bar Graph of {column} Counts')
    axs[i, 1].set_ylabel('Count')  # Assuming you want a y-label for the countplot

# Adjust layout and show plots
plt.tight_layout()
plt.show()

In [ ]:
concatenated_df.drop('Survived', axis=1, inplace=True)
concatenated_df.head(2)

# REGRESSION MODELS 🤖📉

In [ ]:
# Get the index where the train and test data were originally separated
train_data_index = len(train_df)
test_data_index = len(concatenated_df) - len(test_df)

# Split the concatenated data back into train and test sets
X_train = concatenated_df[:train_data_index]
X_test = concatenated_df[test_data_index:]

In [ ]:
# Initialize the models
LogisticRegression_reg = LogisticRegression()
svc_reg = SVC()
KNeighborsClassifier_reg =  KNeighborsClassifier(n_neighbors = 3)
linear_svc = LinearSVC()
decision_tree_reg = DecisionTreeClassifier()
random_forest_reg = RandomForestClassifier(n_estimators=100)

In [ ]:
# Train the models
LogisticRegression_reg.fit(X_train, y_train)
svc_reg.fit(X_train, y_train)
KNeighborsClassifier_reg.fit(X_train, y_train)
linear_svc.fit(X_train, y_train)
decision_tree_reg.fit(X_train, y_train)
random_forest_reg.fit(X_train, y_train)

In [ ]:
# Make predictions using the trained models
LogisticRegression_reg_preds = LogisticRegression_reg.predict(X_test)
svc_reg_preds = svc_reg.predict(X_test)
KNeighborsClassifier_reg_preds = KNeighborsClassifier_reg.predict(X_test)
linear_svc_reg_preds = linear_svc.predict(X_test)
decision_tree_reg_preds = decision_tree_reg.predict(X_test)
random_forest_reg_preds = random_forest_reg.predict(X_test)

In [ ]:
# Create a DataFrame with predictions
predictions_df = pd.DataFrame({
    'Logistic Regression': LogisticRegression_reg_preds,
    'Support Vector Machines': svc_reg_preds,
    'KNN': KNeighborsClassifier_reg_preds,
    'Linear SVC': linear_svc_reg_preds,
    'Decision Tree': decision_tree_reg_preds,
    'Random Forest': random_forest_reg_preds
})

In [ ]:
# Assuming your DataFrame is named 'predictions_df' Calculate the maximum value in each row
max_values = predictions_df.sum(axis=1)

# Set the answer to 1 if the maximum value in the row is 1, otherwise set it to 0
answers = max_values.apply(lambda x: 1 if x >=4  else 0)

In [ ]:
test_ids = sample_submission['PassengerId']

submission_df = pd.DataFrame({'PassengerId': test_ids, 'Survived': answers})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

# Print the head of the submission DataFrame
print(submission_df.head())

# Score: 0.80622 🏆🔢